In [1]:
import pandas as pd
import numpy as np
import requests
import time
import regex as re
from bs4 import BeautifulSoup

In [7]:
links = pd.read_csv('link.csv', header=None)

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept": 
"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US,en;q=0.5", "Accept-Encoding": "gzip, deflate", "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}

articles = []
dates = []
count = 0


for link in links[0] :
    URL = link

    page = requests.get(URL, headers=headers)
    # st.write(page)

    soup = BeautifulSoup(page.content, "html.parser")
    
    # Bitcoin Magazine Scrapper (DONE)
    if 'bitcoinmagazine.com' in URL :
        date = soup.find('time')
        date = str(date)[:42]
        date = re.sub('<time datetime="','',date)
        date = date[:-1]

        text = []
        for data in soup.find_all('p') :
            text.append(data.get_text())
            
        if text :
            dates.append(date)
            articles.append((''.join(text)))
            
        # st.write('Bitcoin Magazine', page, len(text), date)
        print('BitcoinMagazine', page, date, len(text))

    # CoinDesk Scrapper (DONE)
    if 'coindesk.com' in URL :
        date = soup.find_all('span', {'class':'typography__StyledTypography-owin6q-0 dHSCiD'})
        date = re.sub(',|at|UTC', '', date[279].text)
        date = re.sub(' a.m.', ' AM', date)
        date = re.sub(' p.m.', ' PM', date)
        date = re.sub('  ', ' ', date)

        result = soup.find_all('p')

        text = []
        for r in result :
            r = str(r)
            if (r.startswith('<p>')) and (r.endswith('</p>')) :
                r = re.sub('<p>|</p>|<b>|</b>|<a |</a>|</p>|<i>|</i>|<br>|</br>|<br/>|<|>','',r)
                text.append(r)
                
        if text :
            dates.append(date)
            articles.append(''.join(text))

        # st.write('CoinDesk', page, len(text), date)
        print('CoinDesk', page, date, len(text))

    # CoinTelegraph Scrapper
    if 'cointelegraph.com' in URL :
        date = soup.find_all('span', {'class':'typography__StyledTypography-owin6q-0 dHSCiD'})
        date = re.sub(',|at|UTC', '', date[279].text)
        date = re.sub(' a.m.', ' AM', date)
        date = re.sub(' p.m.', ' PM', date)
        date = re.sub('  ', ' ', date)

        text = []
        result = soup.find_all('p', text=True)
        for r in result :
            r = str(r)
            if (r.startswith('<p>')) and (r.endswith('</p>')) :
                r = re.sub('<p>|</p>|<b>|</b>|<a |</p>','',r)
                text.append(r)

        if text :
            articles.append(''.join(text))
            dates.append(date)
        
        # st.write('CoinTelegraph', page, len(text), date)
        print('CoinTelegraph', page, date, len(text))

    # CryptoSlate Scrapper (DONE)
    if 'cryptoslate.com' in URL :
        date = soup.find('span', class_='post-date')
        date = str(date)
        date = re.sub('<span class="post-date">| UTC</span></span>|<span class="time">', '', date)
        date = re.sub('at', '', date)
        
        result = soup.find_all('p')
        text = []
        result = soup.find_all('p')
        for r in result :
            r = str(r)
            r = re.sub('<p>|</p>|<span>|</span>|<p class="post-subheading">|<p class="image-credit">|<a>|</a>', '', str(r))
            r = re.sub('<script>|</script>|[|]', '', r)
            text.append(r)

        if text :
            articles.append(''.join(text))
            dates.append(date)

        # st.write('CryptoSlate', page, len(text), date)
        print('CryptoSlate', page, date, len(text))

    # Yahoo Scrapper (DONE)
    if 'yahoo.com' in URL :
        date = soup.find('time')

        date = str(date)
        date = date.replace('"','')
        date = re.sub('<time class=caas-attr-meta-time datetime=|>|</time>', '',date)
        date = date[:24]
        
        text = []
        result = soup.find_all('p', text=True)
        for r in result :
            r = str(r)
            r = re.sub('"|</p>|<p>','', r)
            if r.startswith('<p class=M(0) C($summaryColor) Fz(14px) Lh(1.43em) LineClamp(3,60px)>') :
                r = r[69:]
            text.append(r)
        
        if text :
            articles.append(''.join(text))
            dates.append(date)

        # st.write('Yahoo', page, len(text), date)
        print('Yahoo', page, date, len(text))

    # Forbes Scrapper (DONE)
    if 'forbes' in URL :
        URL = link
        # Request access to the URL
        page = requests.get(URL, headers=headers)

        # Parse the html using BeautifulSoup for scrapping
        soup = BeautifulSoup(page.content, "html.parser")
        date = soup.find_all('time')
        date = str(date[18]) + str(date[19])
        date = re.sub('<time>|</time>','', date)
        date = re.sub(',',' ', date)
        date = re.sub('  ',' ',date)[:-3]
        # print(date)
        
        text = []
        result = soup.find_all('p')
        for r in result :
            r = str(r)
            r = re.sub('"|</p>|<p>','', r)
            r = re.sub('<p class=color-body light-text>', '',r)
            text.append(r)
        
        if text :
            articles.append(''.join(text))
            dates.append(date)

        # st.write('Forbes', page, len(text), date)
        print('Forbes', page, date, len(text))

    # Nulltx Scrapper (DONE)
    if 'nulltx.com' in URL :
        date = soup.find('time')
        date = re.sub('<time class="entry-date updated td-module-date" datetime="|</time>','',str(date))
        date = date[:25]
        
        text = []
        result = soup.find_all('p')
        for r in result :
            r = str(r)
            r = re.sub('<p>|</p>|<strong>|</strong>', '', r)
            r = re.sub('<p class="comment-form-cookies-consent"><input id="wp-comment-cookies-consent" name="wp-comment-cookies-consent" type="checkbox" value="yes"><label for="wp-comment-cookies-consent">Save my name, email, and website in this browser for the next time I comment.</label></input>','',r)
            r = re.sub('<p class="form-submit"><input class="submit" id="submit" name="submit" type="submit" value="Post Comment"> <input id="comment_post_ID" name="comment_post_ID" type="hidden" value="103352">','',r)
            r = re.sub('<em>Image Source: Vintage Tone/<a href="http://shutterstock.com/" rel="noopener nofollow" target="_blank">Shutterstock.com</a></em>','',r)
            r = re.sub('<input id="comment_parent" name="comment_parent" type="hidden" value="0">|</input></input></input>','',r)
            r = re.sub('nulltx.com is part of the Null Transaction PR media group.','',r)
            r = r.replace('\n\n','')
            text.append(r)

        if text :
            articles.append(''.join(text))
            dates.append(date)

        # st.write('NullTx', page, len(text), date)
        print('NullTx', page, date, len(text))

    # Blockonomi Scrapper (DONE)
    if 'blockonomi.com' in URL :
        date = soup.find('time', class_='post-date')
        date = re.sub('<time class="post-date" datetime="|</time>','',str(date))
        date = date[:25]
        
        text = []
        result = soup.find_all('p')
        for r in result :
            r = str(r)
            r = re.sub('<p>|</p>|<strong>|</strong>|<br>|</br>|<br/>|<em>|</em>|<a>|</a>', '', r)
            r = re.sub('<p class="toc_title">|<p class="text author-bio">|<p class="copyright">','',r)
            text.append(r)

        if text :
            articles.append(''.join(text))
            dates.append(date)

        # st.write('Blockonomi', page, len(text), date)
        print('Blockonomi', page, date, len(text))
    
    
    count += 1

CoinDesk <Response [200]> Aug 4 2021 11:31 AM  2
CoinDesk <Response [200]> Jan 31 2022 11:30 PM  47
CoinDesk <Response [200]> Feb 1 2022 7:55 AM  9
BitcoinMagazine <Response [200]> 2022-02-18T09:10:09-05:00 21
BitcoinMagazine <Response [200]> 2022-02-17T20:00:00-05:00 76
CryptoSlate <Response [200]> Feb. 18, 2022  11:30 am 24
CryptoSlate <Response [200]> Feb. 17, 2022  6:00 pm 35
Yahoo <Response [200]> 2021-06-22T14:06:07.000Z 27
Forbes <Response [200]> Jul 12 2021 08:14am  11
NullTx <Response [200]> 2022-01-22T18:54:40+00:00 16
NullTx <Response [200]> 2022-01-21T22:43:27+00:00 27
Blockonomi <Response [200]> 2022-03-01T16:35:06+00:00 20
Blockonomi <Response [200]> 2022-02-18T11:05:00+00:00 72
Blockonomi <Response [200]> 2022-01-21T11:32:22+00:00 23


In [20]:
links = pd.read_csv('link.csv', header=None)

for link in links[0] :
    print(link)

https://www.coindesk.com/markets/2021/08/04/bitcoin-stuck-below-40k-eyes-short-term-oversold-bounce/
https://www.coindesk.com/markets/2022/01/31/first-mover-asia-crypto-finishes-bad-month-on-high-note/
https://www.coindesk.com/markets/2022/02/01/hackers-move-383m-worth-of-bitcoin-from-2016-bitfinex-hack/
https://bitcoinmagazine.com/business/at-home-bitcoin-mining-is-surging
https://bitcoinmagazine.com/culture/why-should-i-care-about-bitcoin
https://cryptoslate.com/whats-behind-bitcoins-sudden-plunge-to-40000/
https://cryptoslate.com/fans-can-now-use-ethereum-to-tip-creators-on-twitter/
https://finance.yahoo.com/news/forex-dollar-edges-higher-powell-140607366.html
https://www.forbes.com/sites/ninabambysheva/2021/07/12/bitcoin-is-steady-as-it-braces-for-a-big-week/?sh=3a9ac8743ac6
https://nulltx.com/with-bitcoin-and-ethereum-price-crashing-this-web3-coin-gained-over-1800-today/
https://nulltx.com/with-bitcoin-price-crashing-these-3-cryptocurrencies-gained-over-200-today/
https://blockono

In [15]:
len(dates), len(articles)

(14, 14)

In [2]:
#news_df = pd.DataFrame({'Text':articles, 'DateTime':dates})